In [1]:
import os
import sys
import json
import numpy as np
import pandas as pd
import torch
from collections import Counter, defaultdict
from transformers import AutoTokenizer, BasicTokenizer, AutoModelForMaskedLM
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.retrieval.evaluation import EvaluateRetrieval
from pyserini.analysis import Analyzer, get_lucene_analyzer
from tqdm import tqdm_notebook
import pytrec_eval

2023-10-30 21:45:12.942328: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-30 21:45:12.942350: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
analyzer = Analyzer(get_lucene_analyzer())

In [3]:
data_path = "/home/gaia_data/iida.h/BEIR/datasets/nfcorpus"
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/3633 [00:00<?, ?it/s]

In [5]:
# result_path = {"mlm-splade-71694": "/home/gaia_data/iida.h/BEIR/splade_vocab/evaluate/nfcorpus/mlm-splade-idf/71694/analysis.json",
#                "splade": "/home/gaia_data/iida.h/BEIR/splade_vocab/evaluate/nfcorpus/splade-org/analysis.json",
#               "bm25": "/home/gaia_data/iida.h/BEIR/splade_vocab/evaluate/nfcorpus/bm25/analysis.json"}
result_path = {"mlm-splade-71694": "/home/gaia_data/iida.h/BEIR/splade_vocab/evaluate/nfcorpus/mlm-splade-org/71694/analysis.json",
               "mlm-dense-71694": "/home/gaia_data/iida.h/BEIR/splade_vocab/evaluate/nfcorpus/mlm-dense-org/71694/analysis.json",
               "splade": "/home/gaia_data/iida.h/BEIR/splade_vocab/evaluate/nfcorpus/splade-org/analysis.json",
              "bm25": "/home/gaia_data/iida.h/BEIR/splade_vocab/evaluate/nfcorpus/bm25/analysis.json"}

In [6]:
model_pathes = {"mlm-splade-71694": "/home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/",
               "mlm-splade-30522": "/home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/30522/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--30522-batch_size_24-2022-04-11_23-21-18/",
                "splade-71694": "/home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model_init/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-init_model-raw-remove--71694-batch_size_40-2022-04-24_00-46-31/",
               "splade": "/home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_40-2022-05-01_12-37-20"}

In [7]:
tks = {"mlm-splade-71694": AutoTokenizer.from_pretrained(os.path.join(model_pathes["mlm-splade-71694"], "0_MLMTransformer")),
       "splade": AutoTokenizer.from_pretrained(os.path.join(model_pathes["splade"], "0_MLMTransformer")),
      "space": BasicTokenizer(),
      "bm25": analyzer.analyze}

In [8]:
all_results ={}
for k, v in result_path.items():
    with open(v) as fIn:
        all_results[k] = json.load(fIn)

In [9]:
evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'ndcg_cut_10', "P.1"})
all_ndcg = {}
all_ave_ndcg = {}
for k, result in all_results.items():
    if k != "bm25":
        result = list(result.values())[0]
    ndcg = evaluator.evaluate(result)
    all_ndcg[k] = ndcg
    all_ave_ndcg[k] = np.mean([val["ndcg_cut_10"] for val in ndcg.values()])

In [10]:
all_ave_ndcg

{'mlm-splade-71694': 0.34793522628074985,
 'mlm-dense-71694': 0.3276735881003512,
 'splade': 0.3360477401299443,
 'bm25': 0.3352957425731824}

In [14]:
df_all_ndcg = {}
df_all_p1 = {}
# model_names = ["mlm-splade-71694", "splade", "bm25"]
model_names = ["mlm-splade-71694", "mlm-dense-71694", "splade", "bm25"]
qids = set()
for k in model_names:
    target = all_ndcg[k]
    qids |= set(list(target.keys()))
    
for k in model_names:
    target = all_ndcg[k]
    # df_all_ndcg[k] = [val['ndcg_cut_10'] for val in target.values()]
    df_all_ndcg[k] = [target[qid]['ndcg_cut_10'] if qid in target else 0.0 for qid in qids]
    df_all_p1[k] = [target[qid]['P_1'] if qid in target else 0.0 for qid in qids]
    print(len(df_all_ndcg[k]))
    
df_all_ndcg = pd.DataFrame(df_all_ndcg)
df_all_ndcg.index = list(qids)
df_all_ndcg.head()

df_all_p1 = pd.DataFrame(df_all_p1)
df_all_p1.index = list(qids)
df_all_p1.head()


323
323
323
323


,mlm-splade-71694,mlm-dense-71694,splade,bm25
PLAIN-1398,1.0,1.0,1.0,1.0
PLAIN-2880,0.0,0.0,0.0,0.0
PLAIN-3116,0.0,0.0,0.0,0.0
PLAIN-551,1.0,0.0,0.0,0.0
PLAIN-2560,1.0,1.0,1.0,1.0


In [15]:
da_sup = df_all_ndcg[ df_all_ndcg["mlm-splade-71694"] > df_all_ndcg["splade"] + 0.05]
da_sup

,mlm-splade-71694,mlm-dense-71694,splade,bm25
PLAIN-551,1.000000,0.204382,0.000000,0.000000
PLAIN-1109,0.677273,0.701973,0.598395,0.469000
PLAIN-468,0.421421,0.431885,0.301874,0.254882
PLAIN-2680,0.397951,0.098584,0.288200,0.369118
PLAIN-2500,0.114266,0.000000,0.000000,0.000000
...,...,...,...,...
PLAIN-1419,0.510352,0.444097,0.383633,0.432318
PLAIN-2019,0.469279,0.000000,0.181542,0.469279
PLAIN-2177,0.391703,0.538886,0.307015,0.268455
PLAIN-827,0.738192,0.870125,0.672693,0.782739


In [16]:
da_top1 = df_all_p1[(df_all_p1["splade"] < df_all_p1["mlm-splade-71694"]) & (df_all_p1["mlm-splade-71694"] == 1)]
da_top1

,mlm-splade-71694,mlm-dense-71694,splade,bm25
PLAIN-551,1.0,0.0,0.0,0.0
PLAIN-468,1.0,1.0,0.0,0.0
PLAIN-1557,1.0,0.0,0.0,0.0
PLAIN-1667,1.0,1.0,0.0,1.0
PLAIN-1225,1.0,1.0,0.0,0.0
PLAIN-3432,1.0,0.0,0.0,0.0
PLAIN-2620,1.0,1.0,0.0,1.0
PLAIN-2550,1.0,0.0,0.0,0.0
PLAIN-2019,1.0,0.0,0.0,1.0


In [17]:
for qid in da_top1.index:
    print(qid, queries[qid])

PLAIN-551 amnesia
PLAIN-468 Is vitamin D3 (cholecalciferol) preferable to D2 (ergocalciferol)?
PLAIN-1557 magnesium
PLAIN-1667 muscle health
PLAIN-1225 fructose
PLAIN-3432 Healthy Chocolate Milkshakes
PLAIN-2620 Phytates for the Treatment of Cancer
PLAIN-2550 Barriers to Heart Disease Prevention
PLAIN-2019 rickets


In [19]:
sorted(qrels["PLAIN-551"].items(), key=lambda x: -x[1])[:10]

[('MED-4380', 1), ('MED-4381', 1)]

In [14]:
df_all_p1[(df_all_p1["splade"] < df_all_p1["bm25"]) & (df_all_p1["mlm-splade-71694"] == 1)]

,mlm-splade-71694,splade,bm25
PLAIN-1667,1.0,0.0,1.0
PLAIN-2620,1.0,0.0,1.0
PLAIN-2019,1.0,0.0,1.0


In [15]:
da_p1 = df_all_ndcg[(df_all_p1["splade"] < df_all_p1["bm25"]) & (df_all_p1["mlm-splade-71694"] == 1)]
da_p1

,mlm-splade-71694,splade,bm25
PLAIN-1667,0.646402,0.442022,0.742125
PLAIN-2620,0.510584,0.094788,0.626043
PLAIN-2019,0.469279,0.181542,0.469279


In [19]:
qid = "PLAIN-2620"
q = queries[qid]
top_d = sorted(all_results["splade"]["org"][qid].items(), key=lambda x: -x[1])[0][0]
print(corpus[top_d])

print(tks["splade"].tokenize(q))
print(tks["mlm-splade-71694"].tokenize(q))
print(tks["splade"].tokenize(corpus[top_d]["title"]))

{'text': 'Aromatase is a cytochrome P450 enzyme (CYP19) and is the rate limiting enzyme in the conversion of androgens to estrogens. Suppression of in situ estrogen production through aromatase inhibition is the current treatment strategy for hormone-responsive breast cancers. Drugs that inhibit aromatase have been developed and are currently utilized as adjuvant therapy for breast cancer in post-menopausal women with hormone dependent breast cancer. Natural compounds have been studied extensively for important biologic effects such as antioxidant, anti-tumor and anti-viral effects. A significant number of studies have also investigated the aromatase inhibitory properties of a variety of plant extracts and phytochemicals. The identification of natural compounds that inhibit aromatase could be useful both from a chemopreventive standpoint and in the development of new aromatase inhibitory drugs. This review will discuss whole food extracts and the common classes of phytochemicals which 

In [20]:
def calc_idf(corpus, tokenizer):
    N = len(corpus)
    idf = defaultdict(float)
    df = Counter()
    for cid, doc in tqdm_notebook(corpus.items()):
        text = doc["title"] + " " + doc["text"]
        t_doc = tokenizer(text)
        df.update(list(set(t_doc)))
    
    for v, freq in df.items():
        idf[v] = np.log(N/freq)
    return df, idf

def calc_q_idf(queries, tokenizer):
    N = len(queries)
    idf = defaultdict(float)
    df = Counter()
    for qid, query in tqdm_notebook(queries.items()):
        t_q = tokenizer(query)
        df.update(list(set(t_q)))
    
    for v, freq in df.items():
        idf[v] = np.log(N/freq)
    return df, idf

In [21]:
def calc_rsj(index_corpus, queries, qrels, N):
    rsj = {}
    for qid, q_qrels in qrels.items():
        try:
            query = queries[qid]
        except KeyError:
            continue
        t_query = analyzer.analyze(query)
        rel_q_qrels = {k: v for k, v in q_qrels.items() if v > 0}
        Nr = len([i for i in q_qrels.values() if i > 0])
        if Nr < 1:
            continue
        wtq = defaultdict(float)
        for t in t_query:
            Dt = index_corpus[t]
            Nt = len(Dt)
            Ntr = 0
            for did in Dt:
                if did in rel_q_qrels:
                    Ntr+= 1
                   
            pt = Ntr / Nr
            not_pt = (Nt - Ntr) / (N - Nr)
            wtq[t] = np.log(((Ntr + 0.5)*(N - Nt - Nr + Ntr + 0.5))/((Nr - Ntr + 0.5)*(Nt - Ntr + 0.5)) )
            # print(qid, t, N, Nt, Nr, Ntr, wtq[t], ((Ntr + 0.5)*(N - Nt - Nr + Ntr + 0.5)), (Nr - Ntr + 0.5)*(Nt - Ntr + 0.5))
        rsj[qid] = wtq
    return rsj

In [22]:
df, idf = calc_idf(corpus, analyzer.analyze)
N = len(corpus)

/tmp/ipykernel_2900404/2271427616.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cid, doc in tqdm_notebook(corpus.items()):


  0%|          | 0/3633 [00:00<?, ?it/s]

In [23]:
idf_tk = {}
df_tk = {}

for name, tk in tks.items():
    if name != "bm25":
        tokenizer = tk.tokenize
    else:
        tokenizer = tk
    df_, idf_ = calc_idf(corpus, tokenizer)
    idf_tk[name] = idf_
    df_tk[name] = df_

/tmp/ipykernel_2900404/2271427616.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cid, doc in tqdm_notebook(corpus.items()):


  0%|          | 0/3633 [00:00<?, ?it/s]

  0%|          | 0/3633 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/3633 [00:00<?, ?it/s]

  0%|          | 0/3633 [00:00<?, ?it/s]

In [24]:
len(idf_tk["mlm-splade-71694"])

24166

In [22]:
data_path = "/home/gaia_data/iida.h/BEIR/datasets/msmarco"
ms_corpus, ms_queries, ms_qrels = GenericDataLoader(data_folder=data_path).load(split="test")

  0%|          | 0/8841823 [00:00<?, ?it/s]

In [23]:
ms_df, ms_idf = calc_idf(ms_corpus, analyzer.analyze)
ms_N = len(ms_corpus)

/tmp/ipykernel_19344/3501768668.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for cid, doc in tqdm_notebook(corpus.items()):


  0%|          | 0/8841823 [00:00<?, ?it/s]

In [30]:
_, ms_train_queries, _  = GenericDataLoader(data_folder=data_path).load(split="train")

  0%|          | 0/8841823 [00:00<?, ?it/s]

In [33]:
ms_q_df, ms_q_idf = calc_q_idf(ms_train_queries, analyzer.analyze)

/tmp/ipykernel_19344/2271427616.py:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for qid, query in tqdm_notebook(queries.items()):


  0%|          | 0/502939 [00:00<?, ?it/s]

In [34]:
ms_q_N = len(ms_train_queries)
print(ms_q_df["phytat"], ms_q_N)

2 502939


In [42]:
for qid, query in ms_train_queries.items():
    if "phytat" in query.lower():
        print(qid ,query)
        
for did, doc in corpus.items():
    text = doc["title"] + " " + doc["text"]
    if "phytat" in text.lower():
        print(did, end=",")

694789 what is a phytate
783728 what is phytate
MED-2022,MED-2253,MED-2578,MED-2979,MED-2983,MED-2984,MED-2985,MED-2986,MED-2987,MED-2988,MED-2989,MED-4319,

In [24]:
print(ms_df["phytat"], ms_N)
print(np.log((ms_N / ms_df["phytat"])))
print(np.log((len(corpus)/df["phytat"])))

150 8841823
10.984368340976252
5.712907382433201


In [25]:
tks["bm25"]("probiotics")

['probiot']

In [26]:
print(ms_df["probiot"], ms_N, ms_df["probiot"] / ms_N)
print(df["probiot"], N, df["probiot"] / N)
print(np.log((ms_N / ms_df["probiot"])))
print(np.log((len(corpus)/df["probiot"])))

3167 8841823 0.00035818405322069895
27 3633 0.007431874483897605
7.934463588533868
4.901977166216873


In [ ]:
print(len(ms_df))
i_ms_phytat = sorted(ms_df.values()).index(ms_df["phytat"])
i_phytat = sorted(df.values()).index(df["phytat"])
print(i_ms_phytat, i_ms_phytat / len(ms_df))
print(i_phytat, i_phytat / len(df))

In [ ]:
idf_val = np.unique([round(v, 2) for k, v in ms_idf.items()])
list(idf_val).index(round(ms_idf["probiot"], 2)) / len(idf_val)

In [ ]:
q100, q75, q50, q25, q10, q0 = np.percentile(list(ms_idf.values()), [100, 75, 50, 25, 10, 0], interpolation="nearest")
s_ms_idf = sorted(ms_idf.values())
percentile = [int(len(ms_idf) * p) for p in [0.001, 0.05, 0.1, 0.25, 0.5, 0.75, 1]]
print([s_ms_idf[p-1] for p in percentile])

In [ ]:
all_query_term = set()
for query in queries.values():
    t_query = analyzer.analyze(query)
    all_query_term |= set(t_query)
    
index_corpus = defaultdict(list)
for cid, doc in corpus.items():
    text = doc["title"] + " " + doc["text"]
    t_doc = analyzer.analyze(text)
    for t in set(t_doc):
        index_corpus[t].append(cid)

In [ ]:
median_idf = np.median([v for t, v in idf.items() if t in all_query_term] )
q75, q25 = np.percentile(list(idf.values()), [75 ,25])

rsj_u = calc_rsj(index_corpus, queries, qrels, N)
mq75, mq25 = np.percentile([v for qid, term_value in rsj_u.items() for t, v in term_value.items()], [75, 25])
print(median_idf, q75, q25)
print(mq75, mq25)

bin_rsj_u = {}
for t in idf:
    for qid, rsj_uq in rsj_u.items():
        if qid not in bin_rsj_u:
            bin_rsj_u[qid] = {}
        if t not in rsj_uq:
            bin_rsj_u[qid][t] = "OOV"
            continue
        if idf[t] < median_idf:
#             bin_rsj_u[qid][t] = "LowIDF"
            if rsj_uq[t] > mq75:
                bin_rsj_u[qid][t] = "HighRSJ:LowIDF"
            else:
                bin_rsj_u[qid][t] = "LowRSJ:LowIDF"
        else:
#             bin_rsj_u[qid][t] = "HighIDF"
            if rsj_uq[t] > mq75:
                bin_rsj_u[qid][t] = "HighRSJ:HighIDF"
            else:
                bin_rsj_u[qid][t] = "LowRSJ:HighIDF"

In [ ]:
high_rsj_high_idf = set()


for qid, rsj_uq in bin_rsj_u.items():
    for t, bin_ in rsj_uq.items():
        if bin_ == "HighRSJ:HighIDF":
            high_rsj_high_idf.add(t)

In [ ]:
def display(corpus, query, result_qid, qrel_qid, analyzer, top_k, target_term, idf):
    sorted_did_score = sorted(result_qid.items(), key=lambda x: -x[1])[:top_k]
    t_query = analyzer.analyze(query)
    domain_term = []
    for t in t_query:
        if t in target_term:
            domain_term.append(t)
            
    if not domain_term or len(t_query) < 2:
        return
    
    domain_term = {t:round(idf[t], 2) for t in domain_term if t in idf}
            
    print(f"query: {query}, domain_term: {domain_term}")
    for i, (did, score) in enumerate(sorted_did_score):
        qrel = qrel_qid.get(did, -1)
        if qrel > 0 or i == 0:
            print(f"rank: {i+1}, qrel: {qrel}, did: {did}")
            line = corpus[did]
            text = line["title"] + " " + line["text"]
            print(text)

In [ ]:
top_k = 10
for qid in da_sup.index:
    query = queries[qid]
    print(f"===================={qid}===================")
    print(da_sup.loc[qid,:])
    for model in result_path.keys():
        print(f"--------------{model}----------------")  
        if model != "bm25":
            tk_query = tks[model].tokenize(query)
        else:
            tk_query = tks[model](query)
        q_idfs = {t: round(idf_tk[model][t], 2) for t in tk_query if t in idf_tk[model]}
        print(tk_query, q_idfs)
        if model != "bm25":
            result_qid = list(all_results[model].values())[0][qid]
        else:
            try:
                result_qid = all_results[model][qid]
            except:
                qrel_qid = qrels[qid]
                display(corpus, query, result_qid, qrel_qid, analyzer, top_k, high_rsj_high_idf, idf)
                continue
        qrel_qid = qrels[qid]
        display(corpus, query, result_qid, qrel_qid, analyzer, top_k, high_rsj_high_idf, idf)

# Actual Vector

In [29]:
def analysis(model_name, qid, all_results, idf, model_type = "idf", top_k = 10):
    def encode_vec(model, query, e_query, text):
        tok_query = model.tokenizer.tokenize(query)
        tok_text = model.tokenizer.tokenize(text)
        t_text = model.tokenizer(text, max_length=512, return_tensors="pt")
        with torch.no_grad():
            e_text = model.encode(**t_text)
            print(e_query.shape, e_text.shape)
            if model_type == "idf":
                # match_scores = torch.log(1 + e_query * e_text) * idf
                match_scores = e_query * e_text * idf
            else:
                # match_scores = torch.log(1 + e_query * e_text)
                match_scores = e_query * e_text
            wq, qids = torch.topk(e_query, k=20)
            wd, dids = torch.topk(e_text, k=50)
            score, ids = torch.topk(match_scores, k=20)
            ids = ids[0][score[0] > 0.01]
            score = score[0][score[0] > 0.01]
            tqids = model.tokenizer.convert_ids_to_tokens(qids[0])
            tdids = model.tokenizer.convert_ids_to_tokens(dids[0])
            tids = model.tokenizer.convert_ids_to_tokens(ids)
        print([round(s, 2) for s in score.tolist()])
        print(tids)
        print([round(s, 2) for s in wq[0][wq[0] > 0.1].tolist()])
        display(HTML(" ".join([str(t) if t not in tok_query else f'<font color="red">{str(t)}</font>' for t in tqids])))
        print([round(s, 2) for s in wd[0][wd[0] > 0.1].tolist()])
        display_text = []
        for t in tdids:
            if t in tok_query and t in tok_text:
                display_text.append(f'<font color="purple">{t}</font>')
            elif t in tok_query and t not in tok_text:
                display_text.append(f'<font color="blue">{t}</font>')
            elif t in tqids and t in tok_text:
                display_text.append(f'<font color="green">{t}</font>')
            elif t in tqids and t not in tok_text:
                display_text.append(f'<font color="orange">{t}</font>')
            elif t not in tok_text:
                display_text.append(f'<font color="red">{t}</font>')
            else:
                display_text.append(t)
        display(HTML(" ".join(display_text)))
        # print(tdids)
    
    model_path = model_pathes[model_name]
    query = queries[qid]
    top_10_doc = sorted(all_results[model_name][model_type][qid].items(), key=lambda x: -x[1])[:top_k]
    model = Splade(model_path)
    t_query = model.tokenizer(query, max_length=512, return_tensors="pt")
    print(qid, query)
    with torch.no_grad():    
        e_query = model.encode(**t_query)
    for i, (cid, score) in enumerate(top_10_doc):
        try:
            qrel = qrels[qid][cid]
        except:
            qrel = "not-eval"
        print("-------------------")
        print(i, cid, score, qrel)
        text = corpus[cid]["title"] + " " + corpus[cid]["text"]
        encode_vec(model, query, e_query, text)
        
    del model
              

In [30]:
sys.path.append(os.pardir)
from splade_vocab.models  import Splade, BEIRSpladeModel, BEIRSpladeModelIDF
from pprint import pprint
from IPython.display import display, HTML


In [42]:
model_name = "mlm-splade-71694"
this_idf = idf_tk[model_name]
idf_vec = torch.tensor([this_idf[k] if k  in this_idf else 1 for k, v in sorted(tks[model_name].vocab.items(), key=lambda x: x[1])])
analysis(model_name, "PLAIN-2620", all_results, idf_vec, "org", 5)

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


PLAIN-2620 Phytates for the Treatment of Cancer
-------------------
0 MED-2578 18.061906814575195 2
torch.Size([1, 71694]) torch.Size([1, 71694])
[4.04, 3.73, 2.37, 2.33, 2.0, 0.77, 0.61, 0.51, 0.49, 0.42, 0.19, 0.13, 0.13, 0.09, 0.09, 0.08, 0.02, 0.02, 0.02]
['cancer', 'phytate', 'phyto', 'carcinoma', 'ph', '##yt', 'phos', 'treatment', 'drug', 'help', 'is', 'benefit', 'contain', 'phyt', 'effect', 'chemical', 'nutrient', 'vitamin', 'medication']
[2.43, 2.33, 1.88, 1.72, 1.72, 1.53, 1.0, 0.94, 0.83, 0.83, 0.82, 0.76, 0.76, 0.71, 0.63, 0.54, 0.45, 0.42, 0.34, 0.33]


[1.92, 1.9, 1.75, 1.67, 1.66, 1.61, 1.6, 1.6, 1.51, 1.48, 1.45, 1.44, 1.44, 1.43, 1.35, 1.35, 1.31, 1.26, 1.24, 1.2, 1.07, 1.05, 1.05, 1.04, 1.03, 1.02, 1.01, 1.01, 0.99, 0.99, 0.94, 0.93, 0.92, 0.92, 0.9, 0.89, 0.89, 0.88, 0.88, 0.86, 0.86, 0.86, 0.86, 0.85, 0.84, 0.84, 0.84, 0.84, 0.83, 0.81]


-------------------
1 MED-2435 15.862707138061523 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[3.86, 2.45, 2.13, 1.17, 0.98, 0.87, 0.68, 0.54, 0.48, 0.45, 0.36, 0.35, 0.34, 0.26, 0.23, 0.23, 0.15, 0.12, 0.06, 0.05]
['cancer', 'phyto', 'carcinoma', 'phytate', 'treatment', 'ph', 'help', 'drug', '##yt', 'used', 'for', 'treatments', 'cure', 'benefit', 'use', 'is', 'treat', 'effect', 'chemical', 'medication']
[2.43, 2.33, 1.88, 1.72, 1.72, 1.53, 1.0, 0.94, 0.83, 0.83, 0.82, 0.76, 0.76, 0.71, 0.63, 0.54, 0.45, 0.42, 0.34, 0.33]


...

[2.02, 1.84, 1.68, 1.67, 1.65, 1.56, 1.51, 1.42, 1.36, 1.35, 1.34, 1.32, 1.32, 1.28, 1.26, 1.23, 1.2, 1.17, 1.17, 1.16, 1.16, 1.07, 1.06, 1.05, 1.05, 1.02, 1.01, 0.99, 0.98, 0.94, 0.94, 0.94, 0.93, 0.93, 0.92, 0.92, 0.92, 0.9, 0.9, 0.87, 0.87, 0.86, 0.86, 0.83, 0.82, 0.81, 0.81, 0.8, 0.79, 0.78]


-------------------
3 MED-4543 14.681050300598145 1
torch.Size([1, 71694]) torch.Size([1, 71694])
[3.73, 2.01, 1.99, 1.09, 1.08, 0.62, 0.62, 0.48, 0.36, 0.35, 0.33, 0.31, 0.25, 0.24, 0.22, 0.22, 0.22, 0.19, 0.09, 0.08]
['cancer', 'ph', 'carcinoma', 'treatment', 'phyto', 'help', 'drug', 'used', 'is', 'cure', 'for', 'phos', 'treatments', 'dose', 'use', 'effect', 'benefit', 'treat', 'medication', 'contain']
[2.43, 2.33, 1.88, 1.72, 1.72, 1.53, 1.0, 0.94, 0.83, 0.83, 0.82, 0.76, 0.76, 0.71, 0.63, 0.54, 0.45, 0.42, 0.34, 0.33]


[2.17, 1.61, 1.58, 1.58, 1.57, 1.56, 1.54, 1.53, 1.52, 1.45, 1.45, 1.43, 1.42, 1.42, 1.42, 1.33, 1.32, 1.3, 1.24, 1.19, 1.19, 1.18, 1.17, 1.16, 1.16, 1.15, 1.15, 1.15, 1.14, 1.14, 1.14, 1.13, 1.11, 1.1, 1.1, 1.07, 1.06, 1.04, 1.03, 1.03, 1.03, 1.03, 1.03, 1.02, 1.01, 0.99, 0.98, 0.98, 0.98, 0.96]


-------------------
4 MED-4650 14.214611053466797 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[3.54, 1.97, 1.8, 1.41, 0.84, 0.67, 0.65, 0.6, 0.51, 0.4, 0.29, 0.29, 0.25, 0.25, 0.12, 0.12, 0.11, 0.1, 0.09, 0.08]
['cancer', 'carcinoma', 'phyto', 'treatment', 'drug', 'phytate', 'ph', 'help', 'treatments', 'is', '##yt', 'for', 'used', 'cure', 'medication', 'effect', 'treat', 'use', 'benefit', 'contain']
[2.43, 2.33, 1.88, 1.72, 1.72, 1.53, 1.0, 0.94, 0.83, 0.83, 0.82, 0.76, 0.76, 0.71, 0.63, 0.54, 0.45, 0.42, 0.34, 0.33]


[2.29, 1.76, 1.61, 1.49, 1.48, 1.46, 1.43, 1.41, 1.4, 1.33, 1.31, 1.28, 1.28, 1.28, 1.27, 1.22, 1.21, 1.21, 1.17, 1.15, 1.14, 1.14, 1.13, 1.09, 1.07, 1.06, 1.05, 1.05, 1.04, 1.04, 1.03, 1.02, 1.01, 1.0, 1.0, 1.0, 0.99, 0.97, 0.97, 0.97, 0.96, 0.95, 0.95, 0.93, 0.9, 0.88, 0.86, 0.85, 0.84, 0.84]


In [44]:
model_name = 'splade'
analysis(model_name, "PLAIN-2620", all_results, None, "org", 5)

path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_40-2022-05-01_12-37-20


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


PLAIN-2620 Phytates for the Treatment of Cancer
-------------------
0 MED-4650 20.422813415527344 not-eval
torch.Size([1, 30522]) torch.Size([1, 30522])
[5.1, 4.47, 2.64, 2.56, 2.24, 1.34, 0.82, 0.44, 0.31, 0.17, 0.14, 0.11, 0.08, 0.02]
['cancer', '##yt', 'ph', 'tumor', 'treatment', 'for', 'drug', 'therapy', 'function', 'hormone', 'used', 'medication', 'benefit', 'biology']
[2.86, 2.84, 2.26, 2.0, 1.87, 1.83, 1.72, 1.39, 0.78, 0.74, 0.54, 0.46, 0.45, 0.42, 0.36, 0.27, 0.26, 0.26, 0.24, 0.22]


[2.29, 2.04, 2.04, 1.9, 1.79, 1.79, 1.79, 1.64, 1.63, 1.62, 1.57, 1.56, 1.54, 1.41, 1.39, 1.37, 1.37, 1.34, 1.32, 1.3, 1.25, 1.22, 1.22, 1.13, 1.11, 1.11, 1.1, 1.08, 1.05, 1.02, 1.02, 1.02, 0.99, 0.99, 0.98, 0.96, 0.95, 0.95, 0.95, 0.94, 0.93, 0.91, 0.91, 0.89, 0.87, 0.86, 0.84, 0.81, 0.81, 0.8]


-------------------
1 MED-2435 20.205482482910156 not-eval
torch.Size([1, 30522]) torch.Size([1, 30522])
[5.5, 4.63, 2.83, 2.72, 1.5, 1.33, 0.44, 0.4, 0.33, 0.26, 0.15, 0.04, 0.03, 0.03]
['cancer', '##yt', 'ph', 'tumor', 'treatment', 'for', 'therapy', 'drug', 'used', 'function', 'benefit', 'medical', 'of', 'medication']
[2.86, 2.84, 2.26, 2.0, 1.87, 1.83, 1.72, 1.39, 0.78, 0.74, 0.54, 0.46, 0.45, 0.42, 0.36, 0.27, 0.26, 0.26, 0.24, 0.22]


...

[2.06, 1.97, 1.86, 1.86, 1.76, 1.7, 1.61, 1.61, 1.57, 1.56, 1.55, 1.51, 1.49, 1.48, 1.45, 1.43, 1.34, 1.33, 1.3, 1.27, 1.22, 1.22, 1.21, 1.21, 1.19, 1.17, 1.17, 1.14, 1.13, 1.1, 1.09, 1.08, 1.05, 1.04, 1.04, 1.01, 1.01, 1.0, 0.97, 0.97, 0.93, 0.9, 0.88, 0.87, 0.87, 0.86, 0.86, 0.83, 0.82, 0.81]


-------------------
3 MED-2578 19.36952781677246 2
torch.Size([1, 30522]) torch.Size([1, 30522])
[5.38, 5.25, 2.97, 2.49, 2.08, 0.53, 0.4, 0.1, 0.07, 0.05, 0.03, 0.02]
['##yt', 'cancer', 'ph', 'tumor', '##ate', 'for', 'drug', 'benefit', 'of', 'function', 'contain', 'help']
[2.86, 2.84, 2.26, 2.0, 1.87, 1.83, 1.72, 1.39, 0.78, 0.74, 0.54, 0.46, 0.45, 0.42, 0.36, 0.27, 0.26, 0.26, 0.24, 0.22]


[2.27, 2.2, 2.03, 1.89, 1.84, 1.75, 1.74, 1.7, 1.55, 1.55, 1.5, 1.49, 1.45, 1.41, 1.4, 1.36, 1.34, 1.34, 1.33, 1.32, 1.28, 1.23, 1.18, 1.18, 1.15, 1.14, 1.14, 1.1, 1.1, 1.09, 1.08, 1.08, 1.05, 1.05, 1.04, 1.04, 1.04, 1.03, 1.03, 1.01, 0.96, 0.92, 0.92, 0.91, 0.9, 0.9, 0.89, 0.88, 0.88, 0.87]


-------------------
4 MED-4620 18.824283599853516 not-eval
torch.Size([1, 30522]) torch.Size([1, 30522])
[5.42, 4.98, 2.85, 2.61, 1.24, 0.4, 0.31, 0.29, 0.27, 0.22, 0.17, 0.05, 0.01]
['cancer', '##yt', 'ph', 'tumor', 'for', 'therapy', 'function', 'treatment', 'drug', 'species', 'benefit', 'contain', 'biology']
[2.86, 2.84, 2.26, 2.0, 1.87, 1.83, 1.72, 1.39, 0.78, 0.74, 0.54, 0.46, 0.45, 0.42, 0.36, 0.27, 0.26, 0.26, 0.24, 0.22]


[2.39, 2.01, 1.92, 1.9, 1.81, 1.78, 1.75, 1.68, 1.5, 1.45, 1.42, 1.42, 1.4, 1.34, 1.31, 1.26, 1.26, 1.2, 1.16, 1.14, 1.13, 1.13, 1.12, 1.09, 1.08, 1.08, 1.08, 1.06, 1.04, 1.01, 0.99, 0.99, 0.99, 0.98, 0.98, 0.97, 0.97, 0.97, 0.96, 0.93, 0.92, 0.91, 0.9, 0.88, 0.87, 0.85, 0.83, 0.78, 0.78, 0.78]


In [43]:
df["cancer"]

786

In [37]:
corpus['MED-4543']

{'text': 'Phyllanthus emblica Linn. (PE) is a medicinal fruit used in many Asian traditional medicine systems for the treatment of various diseases including cancer. The present study tested the potential anticancer effects of aqueous extract of PE in four ways: (1) against cancer cell lines, (2) in vitro apoptosis, (3) mouse skin tumourigenesis and (4) in vitro invasiveness. The PE extract at 50-100 microg/mL significantly inhibited cell growth of six human cancer cell lines, A549 (lung), HepG2 (liver), HeLa (cervical), MDA-MB-231 (breast), SK-OV3 (ovarian) and SW620 (colorectal). However, the extract was not toxic against MRC5 (normal lung fibroblast). Apoptosis in HeLa cells was also observed as PE extract caused DNA fragmentation and increased activity of caspase-3/7 and caspase-8, but not caspase-9, and up-regulation of the Fas protein indicating a death receptor-mediated mechanism of apoptosis. Treatment of PE extract on mouse skin resulted in over 50% reduction of tumour numbers

In [32]:
model_name = "mlm-splade-71694"
this_idf = idf_tk[model_name]
idf_vec = torch.tensor([this_idf[k] if k  in this_idf else 1 for k, v in sorted(tks[model_name].vocab.items(), key=lambda x: x[1])])
analysis(model_name, "PLAIN-2550", all_results, idf_vec, "org", 5)

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


PLAIN-2550 Barriers to Heart Disease Prevention
-------------------
0 MED-2525 13.86061954498291 2
torch.Size([1, 71694]) torch.Size([1, 71694])
[5.19, 2.61, 1.66, 1.0, 0.94, 0.77, 0.71, 0.18, 0.17, 0.15, 0.11, 0.1, 0.07, 0.07, 0.06, 0.06, 0.05, 0.04, 0.02, 0.01]
['heart', 'prevention', 'disease', 'risk', 'diseases', 'prevent', 'hearts', 'preventive', 'fear', 'importance', 'risks', 'treatment', 'dangerous', 'prophylaxis', 'for', 'contraindications', 'help', 'benefit', 'protect', 'because']
[3.04, 2.5, 2.24, 1.92, 1.88, 1.29, 1.2, 1.16, 1.05, 0.82, 0.7, 0.55, 0.51, 0.51, 0.46, 0.37, 0.34, 0.3, 0.3, 0.3]


[1.97, 1.79, 1.78, 1.71, 1.67, 1.66, 1.54, 1.48, 1.46, 1.36, 1.35, 1.28, 1.28, 1.22, 1.1, 1.1, 1.08, 1.08, 1.07, 1.04, 1.02, 1.0, 0.98, 0.97, 0.97, 0.97, 0.95, 0.93, 0.9, 0.88, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.87, 0.85, 0.85, 0.82, 0.82, 0.82, 0.82, 0.8, 0.79, 0.77, 0.77, 0.76, 0.76, 0.75]


-------------------
1 MED-1679 13.617704391479492 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[4.57, 2.36, 1.6, 1.13, 0.98, 0.82, 0.33, 0.24, 0.21, 0.2, 0.17, 0.14, 0.11, 0.06, 0.05, 0.02, 0.02, 0.01]
['heart', 'prevention', 'disease', 'risk', 'prevent', 'diseases', 'avoid', 'preventive', 'benefit', 'preventable', 'help', 'risks', 'avoidance', 'protect', 'prophylaxis', 'dangerous', 'importance', 'consideration']
[3.04, 2.5, 2.24, 1.92, 1.88, 1.29, 1.2, 1.16, 1.05, 0.82, 0.7, 0.55, 0.51, 0.51, 0.46, 0.37, 0.34, 0.3, 0.3, 0.3]


...

[1.8, 1.74, 1.69, 1.69, 1.61, 1.52, 1.51, 1.37, 1.35, 1.31, 1.26, 1.25, 1.23, 1.18, 1.18, 1.14, 1.13, 1.12, 1.1, 1.06, 1.05, 0.98, 0.97, 0.97, 0.96, 0.93, 0.93, 0.92, 0.9, 0.89, 0.84, 0.83, 0.83, 0.81, 0.81, 0.79, 0.78, 0.78, 0.78, 0.78, 0.77, 0.76, 0.75, 0.75, 0.72, 0.72, 0.69, 0.69, 0.68, 0.68]


-------------------
3 MED-1999 12.951961517333984 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[4.24, 1.94, 1.54, 1.41, 1.06, 0.99, 0.5, 0.25, 0.21, 0.19, 0.14, 0.13, 0.13, 0.11, 0.07, 0.06, 0.04, 0.04, 0.02, 0.02]
['heart', 'prevention', 'disease', 'prevent', 'risk', 'diseases', 'hearts', 'preventive', 'preventable', 'help', 'risks', 'benefit', 'dangerous', 'avoid', 'protect', 'treatment', 'importance', 'consequences', 'because', 'injury']
[3.04, 2.5, 2.24, 1.92, 1.88, 1.29, 1.2, 1.16, 1.05, 0.82, 0.7, 0.55, 0.51, 0.51, 0.46, 0.37, 0.34, 0.3, 0.3, 0.3]


[1.89, 1.63, 1.47, 1.42, 1.4, 1.3, 1.29, 1.28, 1.21, 1.18, 1.17, 1.16, 1.16, 1.14, 1.13, 1.13, 1.09, 1.08, 1.08, 1.05, 1.03, 1.03, 1.02, 1.01, 1.01, 1.01, 1.01, 0.99, 0.99, 0.96, 0.95, 0.95, 0.95, 0.94, 0.94, 0.94, 0.93, 0.91, 0.89, 0.89, 0.89, 0.88, 0.86, 0.83, 0.82, 0.82, 0.82, 0.8, 0.8, 0.78]


-------------------
4 MED-1194 12.584147453308105 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[2.63, 2.41, 1.98, 1.66, 1.44, 0.75, 0.32, 0.29, 0.29, 0.13, 0.11, 0.1, 0.1, 0.1, 0.1, 0.09, 0.04, 0.03, 0.02]
['prevention', 'heart', 'disease', 'diseases', 'prevent', 'risk', 'help', 'preventable', 'preventive', 'avoid', 'protect', 'treatment', 'risks', 'importance', 'dangerous', 'benefit', 'prophylaxis', 'for', 'cause']
[3.04, 2.5, 2.24, 1.92, 1.88, 1.29, 1.2, 1.16, 1.05, 0.82, 0.7, 0.55, 0.51, 0.51, 0.46, 0.37, 0.34, 0.3, 0.3, 0.3]


[1.69, 1.65, 1.61, 1.47, 1.37, 1.3, 1.3, 1.29, 1.26, 1.21, 1.21, 1.08, 1.08, 1.06, 1.05, 1.05, 1.05, 1.05, 1.04, 1.04, 1.01, 1.0, 0.99, 0.98, 0.97, 0.97, 0.96, 0.93, 0.93, 0.92, 0.92, 0.92, 0.91, 0.9, 0.9, 0.9, 0.88, 0.87, 0.86, 0.86, 0.85, 0.82, 0.82, 0.82, 0.81, 0.8, 0.8, 0.79, 0.79, 0.78]


In [34]:
model_name = "mlm-splade-71694"
this_idf = idf_tk[model_name]
idf_vec = torch.tensor([this_idf[k] if k  in this_idf else 1 for k, v in sorted(tks[model_name].vocab.items(), key=lambda x: x[1])])
analysis(model_name, "PLAIN-3432", all_results, idf_vec, "org", 5)

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


PLAIN-3432 Healthy Chocolate Milkshakes
-------------------
0 MED-2225 14.40161418914795 1
torch.Size([1, 71694]) torch.Size([1, 71694])
[6.28, 3.37, 2.89, 0.37, 0.32, 0.3, 0.17, 0.17, 0.09, 0.04, 0.03, 0.03, 0.02, 0.02, 0.01, 0.01, 0.01, 0.01]
['chocolate', 'cocoa', 'healthy', 'eat', 'benefit', 'food', 'foods', 'dose', 'drink', 'flavor', 'amount', 'contain', 'good', 'like', 'treatment', 'calorie', 'brand', 'coating']
[2.76, 2.62, 1.84, 1.65, 1.43, 1.09, 0.95, 0.69, 0.66, 0.63, 0.57, 0.53, 0.48, 0.45, 0.44, 0.39, 0.36, 0.36, 0.35, 0.34]


[2.36, 2.28, 1.87, 1.87, 1.71, 1.64, 1.59, 1.59, 1.48, 1.45, 1.4, 1.4, 1.38, 1.33, 1.27, 1.27, 1.26, 1.22, 1.19, 1.18, 1.18, 1.15, 1.15, 1.14, 1.11, 1.1, 1.08, 1.05, 1.02, 1.02, 1.01, 0.96, 0.95, 0.94, 0.92, 0.91, 0.89, 0.89, 0.88, 0.88, 0.87, 0.87, 0.85, 0.85, 0.82, 0.82, 0.81, 0.8, 0.79, 0.79]


-------------------
1 MED-5283 14.370923042297363 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[6.66, 3.19, 2.3, 0.45, 0.37, 0.34, 0.34, 0.22, 0.18, 0.12, 0.07, 0.04, 0.04, 0.03, 0.02]
['chocolate', 'cocoa', 'healthy', 'eat', 'food', 'benefit', 'good', 'taste', 'foods', 'flavor', 'drink', 'ingredients', 'like', 'contain', 'treatment']
[2.76, 2.62, 1.84, 1.65, 1.43, 1.09, 0.95, 0.69, 0.66, 0.63, 0.57, 0.53, 0.48, 0.45, 0.44, 0.39, 0.36, 0.36, 0.35, 0.34]


...

[2.33, 2.31, 1.76, 1.61, 1.59, 1.58, 1.43, 1.34, 1.29, 1.21, 1.16, 1.16, 1.14, 1.13, 1.11, 1.06, 1.04, 1.03, 0.99, 0.98, 0.97, 0.97, 0.94, 0.94, 0.93, 0.92, 0.88, 0.87, 0.85, 0.84, 0.83, 0.83, 0.82, 0.81, 0.79, 0.79, 0.78, 0.76, 0.76, 0.73, 0.7, 0.7, 0.7, 0.7, 0.68, 0.66, 0.66, 0.66, 0.63, 0.63]


-------------------
3 MED-2227 13.818687438964844 1
torch.Size([1, 71694]) torch.Size([1, 71694])
[5.63, 3.42, 3.1, 0.48, 0.32, 0.31, 0.27, 0.14, 0.06, 0.03, 0.02, 0.02, 0.02]
['chocolate', 'cocoa', 'healthy', 'benefit', 'food', 'eat', 'foods', 'good', 'drink', 'like', 'ingredients', 'treatment', 'contain']
[2.76, 2.62, 1.84, 1.65, 1.43, 1.09, 0.95, 0.69, 0.66, 0.63, 0.57, 0.53, 0.48, 0.45, 0.44, 0.39, 0.36, 0.36, 0.35, 0.34]


[2.39, 2.27, 2.04, 1.91, 1.83, 1.56, 1.55, 1.44, 1.42, 1.36, 1.32, 1.27, 1.23, 1.2, 1.19, 1.15, 1.1, 1.09, 1.09, 1.03, 1.0, 0.99, 0.94, 0.9, 0.88, 0.87, 0.86, 0.86, 0.83, 0.82, 0.8, 0.79, 0.79, 0.79, 0.74, 0.73, 0.72, 0.7, 0.69, 0.68, 0.65, 0.64, 0.64, 0.64, 0.63, 0.62, 0.62, 0.61, 0.6, 0.59]


-------------------
4 MED-5038 13.748722076416016 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[6.23, 3.24, 2.23, 0.68, 0.41, 0.41, 0.3, 0.06, 0.05, 0.04, 0.03, 0.03, 0.02, 0.02]
['chocolate', 'cocoa', 'healthy', 'food', 'foods', 'eat', 'benefit', 'dish', 'drink', 'items', 'like', 'contain', 'flavor', 'good']
[2.76, 2.62, 1.84, 1.65, 1.43, 1.09, 0.95, 0.69, 0.66, 0.63, 0.57, 0.53, 0.48, 0.45, 0.44, 0.39, 0.36, 0.36, 0.35, 0.34]


[2.27, 2.26, 1.67, 1.63, 1.57, 1.28, 1.26, 1.18, 1.08, 1.07, 1.04, 1.04, 0.97, 0.97, 0.96, 0.95, 0.93, 0.88, 0.87, 0.87, 0.86, 0.85, 0.79, 0.78, 0.77, 0.76, 0.75, 0.74, 0.71, 0.7, 0.68, 0.67, 0.66, 0.64, 0.64, 0.61, 0.58, 0.57, 0.54, 0.53, 0.53, 0.49, 0.46, 0.44, 0.43, 0.41, 0.41, 0.41, 0.41, 0.39]


In [33]:
model_name = "mlm-splade-71694"
this_idf = idf_tk[model_name]
idf_vec = torch.tensor([this_idf[k] if k  in this_idf else 1 for k, v in sorted(tks[model_name].vocab.items(), key=lambda x: x[1])])
analysis(model_name, "PLAIN-468", all_results, idf_vec, "org", 5)

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


PLAIN-468 Is vitamin D3 (cholecalciferol) preferable to D2 (ergocalciferol)?
-------------------
0 MED-961 12.258646011352539 2
torch.Size([1, 71694]) torch.Size([1, 71694])
[2.55, 2.27, 1.68, 1.23, 1.03, 0.78, 0.78, 0.58, 0.41, 0.35, 0.24, 0.18, 0.16, 0.13, 0.12, 0.07, 0.06, 0.02, 0.02]
['vitamin', 'd', 'd3', 'than', 'd2', '##3', '##ifer', 'vitamins', 'preferred', 'drug', '3', 'cholecalciferol', 'nutrient', 'comparison', '2', 'chemical', 'do', 'd1', 'd4']
[1.45, 1.35, 1.31, 1.14, 1.01, 0.96, 0.95, 0.87, 0.85, 0.79, 0.78, 0.69, 0.53, 0.52, 0.5, 0.45, 0.41, 0.34, 0.3, 0.29]


[1.76, 1.68, 1.56, 1.48, 1.41, 1.37, 1.36, 1.35, 1.31, 1.3, 1.28, 1.24, 1.23, 1.23, 1.23, 1.16, 1.15, 1.14, 1.13, 1.13, 1.11, 1.08, 1.07, 1.06, 1.06, 1.06, 1.05, 1.02, 1.01, 1.0, 0.99, 0.98, 0.97, 0.94, 0.94, 0.92, 0.92, 0.91, 0.91, 0.91, 0.9, 0.87, 0.86, 0.86, 0.86, 0.86, 0.85, 0.85, 0.84, 0.84]


-------------------
1 MED-4573 12.04101276397705 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[2.99, 2.54, 1.42, 1.1, 0.86, 0.79, 0.56, 0.51, 0.41, 0.37, 0.21, 0.17, 0.08, 0.07, 0.03, 0.01]
['vitamin', 'd', 'd3', 'cholecalciferol', 'erg', 'd2', 'vitamins', '##3', '##ifer', 'er', 'nutrient', 'drug', '3', 'chemical', '2', 'd1']
[1.45, 1.35, 1.31, 1.14, 1.01, 0.96, 0.95, 0.87, 0.85, 0.79, 0.78, 0.69, 0.53, 0.52, 0.5, 0.45, 0.41, 0.34, 0.3, 0.29]


...

[1.88, 1.62, 1.42, 1.32, 1.3, 1.26, 1.21, 1.2, 1.2, 1.19, 1.18, 1.15, 1.11, 1.09, 1.08, 1.04, 1.02, 0.99, 0.98, 0.98, 0.95, 0.95, 0.94, 0.93, 0.93, 0.91, 0.88, 0.88, 0.86, 0.86, 0.85, 0.85, 0.85, 0.84, 0.84, 0.84, 0.81, 0.78, 0.77, 0.76, 0.75, 0.75, 0.75, 0.75, 0.74, 0.7, 0.7, 0.69, 0.69, 0.68]


-------------------
3 MED-3991 11.099977493286133 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[2.36, 2.16, 1.28, 1.26, 1.25, 1.07, 0.67, 0.64, 0.52, 0.39, 0.34, 0.16, 0.11, 0.1, 0.07, 0.03, 0.02]
['vitamin', 'd', 'erg', 'd3', 'cholecalciferol', 'd2', 'er', '##ifer', '##3', 'vitamins', 'drug', 'nutrient', 'with', 'do', 'chemical', 'd1', 'd4']
[1.45, 1.35, 1.31, 1.14, 1.01, 0.96, 0.95, 0.87, 0.85, 0.79, 0.78, 0.69, 0.53, 0.52, 0.5, 0.45, 0.41, 0.34, 0.3, 0.29]


[2.05, 1.99, 1.84, 1.74, 1.62, 1.6, 1.51, 1.51, 1.51, 1.5, 1.41, 1.38, 1.35, 1.34, 1.3, 1.28, 1.28, 1.25, 1.23, 1.18, 1.18, 1.16, 1.14, 1.13, 1.11, 1.11, 1.07, 1.05, 1.05, 1.04, 1.04, 1.03, 1.02, 1.0, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.93, 0.91, 0.89, 0.89, 0.89, 0.88, 0.88, 0.84, 0.84, 0.84]


-------------------
4 MED-3987 10.971348762512207 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[2.57, 2.06, 1.6, 1.09, 0.86, 0.67, 0.56, 0.38, 0.32, 0.28, 0.21, 0.2, 0.12, 0.1, 0.1, 0.05, 0.04, 0.04, 0.03]
['vitamin', 'd', 'd3', 'd2', 'than', '##3', 'vitamins', 'do', 'preferred', 'drug', 'comparison', 'nutrient', 'with', '3', 'cholecalciferol', 'chemical', '2', 'd1', 'd4']
[1.45, 1.35, 1.31, 1.14, 1.01, 0.96, 0.95, 0.87, 0.85, 0.79, 0.78, 0.69, 0.53, 0.52, 0.5, 0.45, 0.41, 0.34, 0.3, 0.29]


[1.77, 1.62, 1.52, 1.4, 1.4, 1.39, 1.36, 1.35, 1.31, 1.22, 1.2, 1.18, 1.16, 1.08, 1.08, 1.07, 1.06, 1.05, 1.03, 1.01, 0.98, 0.98, 0.97, 0.95, 0.89, 0.86, 0.86, 0.85, 0.82, 0.81, 0.81, 0.8, 0.78, 0.78, 0.77, 0.76, 0.76, 0.75, 0.75, 0.73, 0.73, 0.73, 0.69, 0.69, 0.67, 0.66, 0.66, 0.66, 0.66, 0.65]


In [38]:
model_name = "mlm-splade-71694"
this_idf = idf_tk[model_name]
idf_vec = torch.tensor([this_idf[k] if k  in this_idf else 1 for k, v in sorted(tks[model_name].vocab.items(), key=lambda x: x[1])])
analysis(model_name, "PLAIN-551", all_results, idf_vec, "org", 5)

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


PLAIN-551 amnesia
-------------------
0 MED-4381 8.197067260742188 1
torch.Size([1, 71694]) torch.Size([1, 71694])
[4.01, 2.23, 0.54, 0.38, 0.31, 0.29, 0.23, 0.11, 0.09]
['amnesic', 'amnesia', 'symptoms', 'is', 'phenomenon', 'cause', 'effect', 'memory', 'definition']
[2.9, 1.99, 0.92, 0.71, 0.63, 0.59, 0.55, 0.52, 0.42, 0.38, 0.34, 0.33, 0.31, 0.28, 0.26, 0.24, 0.24, 0.15, 0.14, 0.11]


[2.01, 1.89, 1.85, 1.72, 1.69, 1.57, 1.5, 1.46, 1.42, 1.41, 1.37, 1.36, 1.36, 1.28, 1.28, 1.26, 1.26, 1.25, 1.22, 1.21, 1.17, 1.17, 1.17, 1.15, 1.15, 1.14, 1.13, 1.08, 1.07, 1.06, 1.0, 0.98, 0.96, 0.91, 0.87, 0.86, 0.84, 0.83, 0.83, 0.83, 0.82, 0.81, 0.79, 0.79, 0.77, 0.76, 0.76, 0.75, 0.73, 0.73]


-------------------
1 MED-4380 4.489853382110596 1
torch.Size([1, 71694]) torch.Size([1, 71694])
[2.9, 0.51, 0.3, 0.25, 0.22, 0.2, 0.09]
['amnesic', 'is', 'phenomenon', 'symptoms', 'effect', 'cause', 'memory']
[2.9, 1.99, 0.92, 0.71, 0.63, 0.59, 0.55, 0.52, 0.42, 0.38, 0.34, 0.33, 0.31, 0.28, 0.26, 0.24, 0.24, 0.15, 0.14, 0.11]


...

[1.9, 1.72, 1.65, 1.64, 1.38, 1.22, 1.21, 1.21, 1.17, 1.17, 1.13, 1.13, 1.11, 1.1, 1.09, 1.08, 1.07, 1.05, 1.04, 1.04, 1.03, 1.01, 0.96, 0.93, 0.92, 0.91, 0.91, 0.9, 0.9, 0.88, 0.86, 0.86, 0.85, 0.84, 0.81, 0.8, 0.8, 0.8, 0.79, 0.78, 0.77, 0.77, 0.77, 0.76, 0.76, 0.76, 0.76, 0.75, 0.74, 0.73]


-------------------
3 MED-4789 2.959315299987793 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[2.31, 0.28, 0.11, 0.04]
['amnesic', 'effect', 'disability', 'memory']
[2.9, 1.99, 0.92, 0.71, 0.63, 0.59, 0.55, 0.52, 0.42, 0.38, 0.34, 0.33, 0.31, 0.28, 0.26, 0.24, 0.24, 0.15, 0.14, 0.11]


[2.03, 1.89, 1.85, 1.57, 1.54, 1.47, 1.41, 1.37, 1.34, 1.33, 1.3, 1.29, 1.28, 1.25, 1.25, 1.25, 1.25, 1.24, 1.22, 1.19, 1.18, 1.16, 1.16, 1.14, 1.12, 1.11, 1.06, 1.03, 1.02, 1.02, 1.01, 1.01, 0.99, 0.99, 0.98, 0.97, 0.95, 0.93, 0.91, 0.91, 0.9, 0.88, 0.88, 0.85, 0.83, 0.83, 0.82, 0.82, 0.82, 0.81]


-------------------
4 MED-5002 2.8325138092041016 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[0.92, 0.72, 0.23, 0.21, 0.11, 0.07, 0.06]
['amnesia', 'amnesic', 'memory', 'effect', 'cause', 'disability', 'is']
[2.9, 1.99, 0.92, 0.71, 0.63, 0.59, 0.55, 0.52, 0.42, 0.38, 0.34, 0.33, 0.31, 0.28, 0.26, 0.24, 0.24, 0.15, 0.14, 0.11]


[2.08, 1.8, 1.76, 1.74, 1.68, 1.68, 1.67, 1.65, 1.63, 1.59, 1.58, 1.55, 1.54, 1.52, 1.49, 1.29, 1.26, 1.26, 1.26, 1.25, 1.24, 1.21, 1.21, 1.21, 1.2, 1.19, 1.18, 1.14, 1.13, 1.09, 1.09, 1.06, 1.06, 1.04, 1.04, 1.03, 1.03, 1.03, 1.02, 1.0, 0.99, 0.98, 0.98, 0.97, 0.97, 0.95, 0.95, 0.92, 0.91, 0.9]


In [40]:
corpus["MED-4381"]

{'text': 'Amnesic shellfish poisoning (ASP) is caused by consumption of shellfish that have accumulated domoic acid, a neurotoxin produced by some strains of phytoplankton. The neurotoxic properties of domoic acid result in neuronal degeneration and necrosis in specific regions of the hippocampus. A serious outbreak of ASP occurred in Canada in 1987 and involved 150 reported cases, 19 hospitalisations and 4 deaths after consumption of contaminated mussels. Symptoms ranged from gastrointestinal disturbances, to neurotoxic effects such as hallucinations, memory loss and coma. Monitoring programmes are in place in numerous countries worldwide and closures of shellfish harvesting areas occur when domoic acid concentrations exceed regulatory limits. This paper reviews the chemistry, sources, metabolism and toxicology of domoic acid as well as human case reports of ASP and discusses a possible mechanism of toxicity.',
 'title': 'Amnesic shellfish poison.'}

In [39]:
model_name = 'splade'
analysis(model_name, "PLAIN-551", all_results, None, "org", 5)

path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_40-2022-05-01_12-37-20


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


PLAIN-551 amnesia
-------------------
0 MED-4860 4.507206916809082 not-eval
torch.Size([1, 30522]) torch.Size([1, 30522])
[1.55, 1.36, 0.88, 0.52, 0.12, 0.05, 0.02]
['amnesia', 'memory', 'alzheimer', 'memories', 'symptoms', 'treatment', 'effect']
[2.99, 1.91, 1.07, 0.81, 0.72, 0.66, 0.63, 0.54, 0.54, 0.47, 0.38, 0.38, 0.24, 0.21]


[2.39, 1.96, 1.89, 1.89, 1.7, 1.64, 1.59, 1.53, 1.44, 1.44, 1.43, 1.43, 1.41, 1.41, 1.35, 1.35, 1.33, 1.29, 1.23, 1.22, 1.21, 1.21, 1.12, 1.11, 1.1, 1.1, 1.07, 1.05, 1.04, 1.04, 1.03, 1.02, 1.01, 1.01, 0.99, 0.97, 0.96, 0.95, 0.94, 0.94, 0.92, 0.92, 0.89, 0.87, 0.87, 0.87, 0.86, 0.85, 0.85, 0.85]


-------------------
1 MED-4996 3.967149257659912 not-eval
torch.Size([1, 30522]) torch.Size([1, 30522])
[1.8, 1.44, 0.54, 0.21, 0.03, 0.01]
['amnesia', 'memory', 'memories', 'alzheimer', 'effect', 'treatment']
[2.99, 1.91, 1.07, 0.81, 0.72, 0.66, 0.63, 0.54, 0.54, 0.47, 0.38, 0.38, 0.24, 0.21]


...

[3.0, 2.19, 1.92, 1.88, 1.82, 1.8, 1.64, 1.51, 1.5, 1.45, 1.42, 1.35, 1.29, 1.26, 1.24, 1.23, 1.2, 1.18, 1.17, 1.11, 1.11, 1.1, 1.09, 1.06, 1.06, 1.04, 1.03, 1.02, 0.97, 0.97, 0.96, 0.94, 0.94, 0.93, 0.92, 0.92, 0.89, 0.89, 0.87, 0.86, 0.85, 0.84, 0.81, 0.81, 0.8, 0.77, 0.77, 0.77, 0.76, 0.76]


-------------------
3 MED-5002 3.883087158203125 not-eval
torch.Size([1, 30522]) torch.Size([1, 30522])
[1.3, 1.09, 0.78, 0.49, 0.02]
['memory', 'amnesia', 'alzheimer', 'memories', 'effect']
[2.99, 1.91, 1.07, 0.81, 0.72, 0.66, 0.63, 0.54, 0.54, 0.47, 0.38, 0.38, 0.24, 0.21]


[2.61, 2.4, 1.88, 1.81, 1.74, 1.7, 1.62, 1.61, 1.57, 1.53, 1.52, 1.47, 1.45, 1.45, 1.43, 1.43, 1.38, 1.38, 1.37, 1.36, 1.34, 1.28, 1.28, 1.26, 1.26, 1.26, 1.23, 1.23, 1.22, 1.21, 1.21, 1.19, 1.19, 1.18, 1.17, 1.15, 1.15, 1.14, 1.13, 1.13, 1.12, 1.09, 1.05, 1.05, 1.03, 1.01, 1.0, 0.99, 0.98, 0.98]


-------------------
4 MED-1497 3.763014316558838 not-eval
torch.Size([1, 30522]) torch.Size([1, 30522])
[1.58, 0.59, 0.57, 0.36, 0.33, 0.2, 0.12, 0.01]
['alzheimer', 'phenomenon', 'disease', 'symptoms', 'memory', 'syndrome', 'meaning', 'effect']
[2.99, 1.91, 1.07, 0.81, 0.72, 0.66, 0.63, 0.54, 0.54, 0.47, 0.38, 0.38, 0.24, 0.21]


[2.68, 2.39, 2.01, 1.83, 1.78, 1.71, 1.71, 1.7, 1.65, 1.63, 1.51, 1.44, 1.43, 1.43, 1.36, 1.36, 1.34, 1.33, 1.26, 1.25, 1.24, 1.22, 1.2, 1.19, 1.18, 1.16, 1.14, 1.14, 1.11, 1.11, 1.09, 1.08, 1.07, 1.06, 1.04, 1.01, 1.0, 0.97, 0.92, 0.92, 0.91, 0.89, 0.89, 0.87, 0.86, 0.85, 0.84, 0.83, 0.77, 0.76]


In [41]:
corpus["MED-4860"]

{'text': 'The prevalence of dementia is increasing with expansion of the older adult population. In the absence of effective therapy, preventive approaches are essential to address this public health problem. Blueberries contain polyphenolic compounds, most prominently anthocyanins, which have antioxidant and anti-inflammatory effects. In addition, anthocyanins have been associated with increased neuronal signaling in brain centers mediating memory function as well as improved glucose disposal, benefits that would be expected to mitigate neurodegeneration. We investigated the effects of daily consumption of wild blueberry juice in a sample of nine older adults with early memory changes. At 12 weeks, we observed improved paired associate learning (p = 0.009) and word list recall (p = 0.04). In addition, there were trends suggesting reduced depressive symptoms (p = 0.08) and lower glucose levels (p = 0.10). We also compared the memory performances of the blueberry subjects with a demogra

In [73]:
model_name = "mlm-splade-71694"
this_idf = idf_tk[model_name]
idf_vec = torch.tensor([this_idf[k] if k  in this_idf else 1 for k, v in sorted(tks[model_name].vocab.items(), key=lambda x: x[1])])
analysis(model_name, "PLAIN-418", all_results, idf_vec, "org", 1)

path /home/gaia_data/iida.h/BEIR/model/pubmed_abst/bert-base-uncased/splade_model/raw/remove/71694/distilSplade_0.1_0.08_-groups-gcb50243-iida.h-BEIR-model-pubmed_abst-bert-base-uncased-mlm_model-raw-remove--71694-batch_size_40-2022-04-12_08-52-34/


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


PLAIN-418 Fresh fruit versus frozen--which is better?
-------------------
0 MED-3738 10.327356338500977 not-eval
torch.Size([1, 71694]) torch.Size([1, 71694])
[2.11, 1.71, 1.53, 1.19, 0.93, 0.83, 0.81, 0.75, 0.3, 0.2, 0.15, 0.13, 0.1, 0.08, 0.05]
['fruit', 'freezing', 'fresh', 'fruits', 'benefit', 'freeze', 'than', 'frozen', 'juice', 'refrigerator', 'food', 'grapes', 'foods', 'comparison', 'do']
[1.92, 1.84, 1.83, 1.81, 1.48, 1.45, 1.35, 1.22, 1.18, 1.03, 0.97, 0.68, 0.64, 0.62, 0.55, 0.46, 0.33, 0.33, 0.31, 0.3]


[1.83, 1.77, 1.71, 1.6, 1.56, 1.51, 1.51, 1.46, 1.43, 1.42, 1.39, 1.34, 1.31, 1.3, 1.3, 1.28, 1.24, 1.22, 1.22, 1.17, 1.15, 1.14, 1.13, 1.1, 1.1, 1.09, 1.08, 1.08, 1.07, 1.06, 1.05, 1.03, 1.03, 1.01, 1.0, 0.99, 0.96, 0.96, 0.96, 0.94, 0.94, 0.93, 0.92, 0.92, 0.92, 0.91, 0.91, 0.91, 0.91, 0.9]


In [ ]:
model_name = 'splade'
analysis(model_name, "PLAIN-418", all_results, None, "org", 2)

In [280]:
model_name = 'splade'
analysis(model_name, "PLAIN-153", all_results, None, "org", 20)

path /home/gaia_data/iida.h/BEIR/model/msmarco/splade/distilSplade_0.1_0.08_bert-base-uncased-batch_size_40-2022-05-01_12-37-20


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


-------------------
0 MED-3692 16.833602905273438 1
torch.Size([1, 30522]) torch.Size([1, 30522])
[2.02, 1.87, 1.24, 0.54, 0.44, 0.39, 0.27, 0.19, 0.07]
['##biotic', 'pro', 'antibiotics', 'bacteria', '##s', 'drug', 'for', 'use', 'treatment']
[2.94, 2.53, 2.06, 2.04, 1.86, 1.8, 1.43, 1.06, 0.85, 0.77, 0.74, 0.74, 0.67, 0.43, 0.42, 0.33, 0.26, 0.25, 0.17, 0.15]
['##biotic', 'pro', 'take', 'to', 'should', 'antibiotics', 'you', 'drug', 'must', 'how', 'use', 'bacteria', '##s', 'dose', 'for', 'best', 'diet', 'need', 'stay', 'eat']
[2.22, 2.21, 2.17, 1.68, 1.63, 1.5, 1.48, 1.38, 1.3, 1.25, 1.21, 1.17, 1.13, 1.12, 1.11, 1.07, 1.04, 1.03, 0.98, 0.98, 0.98, 0.97, 0.96, 0.95, 0.93, 0.91, 0.91, 0.89, 0.87, 0.85, 0.84, 0.82, 0.8, 0.78, 0.78, 0.77, 0.76, 0.75, 0.75, 0.74, 0.7, 0.7, 0.68, 0.68, 0.66, 0.66, 0.66, 0.64, 0.64, 0.61]
['##biotic', 'gut', 'pro', 'micro', 'random', '##bio', 'isa', 'antibiotics', 'human', 'metabolic', '##enter', 'pre', '##ta', 'ag', 'live', 'controlled', '##estinal', '##pp',

In [281]:
corpus["MED-1006"]

{'text': 'Functional abdominal pain in the context of irritable bowel syndrome (IBS) is a challenging problem for primary care physicians, gastroenterologists and pain specialists. We review the evidence for the current and future non-pharmacological and pharmacological treatment options targeting the central nervous system and the gastrointestinal tract. Cognitive interventions such as cognitive behavioral therapy and hypnotherapy have demonstrated excellent results in IBS patients, but the limited availability and labor-intensive nature limit their routine use in daily practice. In patients who are refractory to first-line therapy, tricyclic antidepressants (TCA) and selective serotonin reuptake inhibitors are both effective to obtain symptomatic relief, but only TCAs have been shown to improve abdominal pain in meta-analyses. A diet low in fermentable carbohydrates and polyols (FODMAP) seems effective in subgroups of patients to reduce abdominal pain, bloating, and to improve the st

In [332]:
_, ms_queries_train, _ = GenericDataLoader(data_folder=data_path).load(split="train")

  0%|          | 0/8841823 [00:00<?, ?it/s]

In [335]:
num_probiot = []
for qid, txt in ms_queries_train.items():
    if "probiotics" in txt:
        num_probiot.append(txt.lower())

len(num_probiot)

55

In [336]:
len(ms_queries_train)

502939

In [340]:
word_freq = Counter()
for qid, txt in ms_queries_train.items():
    word_freq.update(tks["bm25"](txt))

In [350]:
word_rank = {k: (i, v) for i, (k, v) in enumerate(word_freq.most_common())}

In [354]:
print(len(word_freq))
print(word_rank["probiot"])

80279
(3248, 78)


In [363]:
freq = {v: [] for v  in word_freq.values()}
for k, v in word_freq.items():
    freq[v].append(k)
    
freq_rank = {k: i for i, (k, v) in enumerate(freq.items())}

In [365]:
freq_rank[word_freq["probiot"]]

143

In [366]:
len(freq_rank)

943

In [373]:
sorted(freq.items(), reverse=True) [:10]

[(201743, ['what']),
 (72760, ['how']),
 (44511, ['doe']),
 (26407, ['do']),
 (22971, ['definit']),
 (22748, ['where']),
 (20780, ['mean']),
 (20190, ['who']),
 (18871, ['long']),
 (17838, ['when'])]

In [381]:
tks["bm25"]("eubacterium")

['eubacterium']

In [382]:
word_freq["eubacterium"]

0

In [383]:
ms_idf["eubacterium"]

12.816949804724562